<a href="https://colab.research.google.com/github/ptmdmusique/cs510-data-with-python/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What Da Music

## Description

- Analyze and learn about music created over the years
- Predict next year trend

## Target questions

* What's the distribution of different genre and other interesting columns?
* Is there a correlation between tempo and popularity of a song?
* Do your most repeated songs fall into a specific mood cluster (grouped by valence)?
* How do moods differ between genres (e.g.: rap vs classical)?
* What's the popularity over the year (e.g.: based on genre)?


## Data fetching

Fetch data from Hugging Face. The sources are inside the cell itself.
There are 2 dataset that will be used for this project
* Detail song dataset - [link](https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset) - contains information such as the liveliness, popularity, etc of the songs
* Song metadata dataset - [link](https://huggingface.co/datasets/bigdata-pw/Spotify) - contains songs' metadata such as published year

However, to avoid rate limiting, the data loaded from a local csv and a zip folder instead

In [ ]:
import pandas as pd
import dask.dataframe as dd
!pip install -q gdown
import gdown
import ast

In [ ]:
# Load from dataset
# main_df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

# Load locally
gdown.download("https://drive.google.com/uc?id=17of8pv_Al00Er66cewY57NAu9BSDIvHy", "main_data.csv", quiet=False)
main_df = pd.read_csv("main_data.csv")
main_df.head()

Downloading...
From: https://drive.google.com/uc?id=17of8pv_Al00Er66cewY57NAu9BSDIvHy
To: /content/main_data.csv
100%|██████████| 20.1M/20.1M [00:00<00:00, 160MB/s]


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
# Load from dataset
# metadata_df = dd.read_parquet("hf://datasets/bigdata-pw/Spotify/**/*.parquet")

# Load locally
gdown.download("https://drive.google.com/uc?id=1zOK8dgeKFkAOB1XfvPHQXIumJHQ-Mwxg", "metadata.zip", quiet=False)
!unzip metadata.zip -d metadata
metadata_df = dd.read_csv("metadata/metadata/metadata_*.csv")

metadata_df.head()

,album_name,album_uri,album_date,artists,content_rating,duration,language,name,playcount,popularity,uri
0,Glazunov: Symphony No. 1 & Violin Concerto,6UuhdubAi7oWtZe8uK2hwP,"{'day': 1.0, 'month': 9.0, 'year': 1999.0}",[{'artist_gid': 'ea7c8f0f20304977805a2a06d305b...,NONE,677440.0,['zxx'],"Violin Concerto in A Minor, Op. 82: II. Andante",0.0,1.0,00qxNz44liU0GuNvU6jK3S
1,Mississippi,1F0vVZSBGEwQCQLpMNAoX8,"{'day': 11.0, 'month': 8.0, 'year': 2023.0}",[{'artist_gid': 'ea8667fa63fd40919571ff6702463...,NONE,184514.0,['en'],Whistle,18327.0,17.0,7tMBIFLRHcwiwFSqTGr4nm
2,From Our Heart,5WRdAyymmUO5HTsXVd7Ki9,"{'day': 11.0, 'month': 8.0, 'year': 1988.0}",[{'artist_gid': 'ff2ec8aa0d6c461bb97aadf34655f...,NONE,177476.0,['en'],Jerusalem Coming Down,0.0,0.0,1Y126hnIT64R1nbqRiHGjY
3,Thunder Meditation,2N37AZizDFyq8uT1HqpIbt,"{'day': 23.0, 'month': 11.0, 'year': 2018.0}",[{'artist_gid': '5bb72b6b447d4ac9a61aaa3f42e47...,NONE,226559.0,['zxx'],Finnish Rain,5482033.0,22.0,3XjsUzQH4y0EL7PhMSnlSs
4,Marathia,3AjVuMTqsXGR7vXNo9OwSu,"{'day': 28.0, 'month': 10.0, 'year': 2022.0}",[{'artist_gid': 'a7492507f9cc49b2b470c33c51bad...,NONE,133810.0,['zxx'],Marathia,2808098.0,33.0,6Vcmffnur4CAIFaV0Z1VHv


## Data Cleaning

As we're analyzing the song trend over the years, we need to have the year column appended to the main dataset
The goal of this section is to
* Filter out only the rows of the main dataset that has the full data in the metadata dataset
* Drop any nonsense column
* Do statistical analysis to remove outliers (e.g.: song that were published too early)

### Get only the rows that has metadata

In [ ]:
uri_df = metadata_df[['uri']].compute()
uri_set = set(uri_df['uri'].dropna()) # Shove into a set for super duper fast lookup

rows_with_full_data_mask = main_df['track_id'].isin(uri_set)
filtered_main_df = main_df[rows_with_full_data_mask]

print("Main shape", main_df.shape)
print("Filtered shape", filtered_main_df.shape)

filtered_main_df.to_csv("filtered_tracks.csv", index=False)

Main shape (114000, 21)
Filtered shape (70733, 21)


### Join the metadata with the main data

In [ ]:
track_ids = set(filtered_main_df['track_id'].dropna())
track_id_mask = metadata_df['uri'].isin(track_ids)

# The original data has too many data that we'll not used
# So filter by only the valid entries to speed up the process, urg 🏃‍♀️
filtered_metadata_df = metadata_df[track_id_mask][['uri', 'album_date', 'language', 'playcount']]

# We don't really care about the other columns as they don't contain useful values or we already have it in the main df
metadata_subset = filtered_metadata_df.compute()

merged_df = filtered_main_df.merge(
    metadata_subset,
    left_on='track_id',
    right_on='uri',
    how='inner'
).drop(columns=['uri']) # Also drop the dup column

In [ ]:
## Or just load the pre-computed file
# gdown.download("https://drive.google.com/uc?id=1WXbU9DvBwcTiRntsZbkA9tPPpCNSq4uF", "merged_data.csv", quiet=False)
# merged_df = pd.read_csv("merged_data.csv")

In [ ]:
# merged_df.to_csv("merged_data.csv", index=False)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70733 entries, 0 to 70732
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        70733 non-null  int64  
 1   track_id          70733 non-null  object 
 2   artists           70733 non-null  object 
 3   album_name        70733 non-null  object 
 4   track_name        70733 non-null  object 
 5   popularity        70733 non-null  int64  
 6   duration_ms       70733 non-null  int64  
 7   explicit          70733 non-null  bool   
 8   danceability      70733 non-null  float64
 9   energy            70733 non-null  float64
 10  key               70733 non-null  int64  
 11  loudness          70733 non-null  float64
 12  mode              70733 non-null  int64  
 13  speechiness       70733 non-null  float64
 14  acousticness      70733 non-null  float64
 15  instrumentalness  70733 non-null  float64
 16  liveness          70733 non-null  float6

### Drop nonsense columns

In [ ]:
column_to_drop = ['Unnamed: 0']
merged_df.drop(columns=column_to_drop, inplace=True, errors='ignore')

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70733 entries, 0 to 70732
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          70733 non-null  object 
 1   artists           70733 non-null  object 
 2   album_name        70733 non-null  object 
 3   track_name        70733 non-null  object 
 4   popularity        70733 non-null  int64  
 5   duration_ms       70733 non-null  int64  
 6   explicit          70733 non-null  bool   
 7   danceability      70733 non-null  float64
 8   energy            70733 non-null  float64
 9   key               70733 non-null  int64  
 10  loudness          70733 non-null  float64
 11  mode              70733 non-null  int64  
 12  speechiness       70733 non-null  float64
 13  acousticness      70733 non-null  float64
 14  instrumentalness  70733 non-null  float64
 15  liveness          70733 non-null  float64
 16  valence           70733 non-null  float6

### Drop outliers

Drop the songs that's too outdated

The `album_date` is actually a string column representing jsons. So we'll first need to convert that into an actual json column first

An example value
```json
  {
    "day": 1,
    "month": 9,
    "year": 1999
  }
  
```

In [ ]:
album_date_json_df = merged_df['album_date'].apply(ast.literal_eval)

NameError: name 'merged_df' is not defined

In [ ]:
merged_df['album_year'] = album_date_json_df.apply(lambda x: print(type(x)))
merged_df['album_year']

## Data transformation

* Group data into liveliness groups based on `valence` column and an arbitrary heuristic criteria